## 1. Importing the libraries

In [81]:
import cv2
import pandas as pd
from matplotlib.pyplot import imshow 
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm
import numpy as np
%matplotlib inline
import datetime, os, time
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import seaborn as sns
import shutil
from PIL import Image
os.chdir("/home/kanaad/Desktop/Strathclyde/Project/IchMLP/HHP/simulation/cropped pas stained images hhp - 2/")

## 2. Reading the Schizont file

In [82]:
excel = pd.read_excel('Schizont measurements.xlsx')

In [83]:
excel.head()

,ID,Sample,No.,No. (target area),Area,Unit,Perimeter,Unit.1,Major axis,Unit.2,...,Unnamed: 14,Unnamed: 15,THIS DATA HAS BEEN DOUBLE CHECKED FOR ACCURACY,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,1,9-1 6,1,1,183,µm2,50,µm,19,µm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,9-1 6,2,1,1670,µm2,152,µm,51,µm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,9-1 7,1,1,1295,µm2,137,µm,47,µm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,9-1 7,2,1,2029,µm2,164,µm,56,µm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,9-1 9,1,1,1063,µm2,134,µm,42,µm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
excel = excel.drop(excel.columns[14:],axis=1)

## 3. Aggregating the data

In [85]:
area_sets = {}
area_sets['file_name'] = {'area': 10, 'count': 5 }
for i in tqdm(range(len(excel))):
    file_name = excel['Sample'].iloc[i]
    area = excel['Area'].iloc[i]
    
    if file_name in area_sets:
        area_sets[file_name]['count'] = area_sets[file_name]['count'] + excel['No. (target area)'].iloc[i]
        area_sets[file_name]['area'] = area_sets[file_name]['area']+ area
    else:
        area_sets[file_name] = {}
        area_sets[file_name]['count'] = excel['No. (target area)'].iloc[i]
        area_sets[file_name]['area'] = area

100%|██████████| 7538/7538 [00:00<00:00, 29560.08it/s]


In [86]:
excel_dataframe = pd.DataFrame.from_dict(area_sets,orient='index')
excel_dataframe = excel_dataframe.drop(excel_dataframe.index[[0]])
excel_dataframe.to_csv('area_count_agg_2.csv')

In [87]:
excel_dataframe = pd.read_csv('area_count_agg_2.csv')
excel_dataframe = excel_dataframe.rename(columns={'Unnamed: 0':'file_name'})

## 4. Cleaning the dataframe filenames

In [88]:
for i in tqdm(range(len(excel_dataframe))):
    orig = excel_dataframe['file_name'].iloc[i]
    new = orig.replace(" ","")
    excel_dataframe['file_name'].iloc[i] = new

100%|██████████| 442/442 [00:00<00:00, 6674.50it/s]


## 4.1 modifying the file names

In [89]:
excel_dataframe['modified_file'] = 0
for i in tqdm(range(len(excel_dataframe))):
    area = excel_dataframe['area'].iloc[i]
    sci_count = excel_dataframe['count'].iloc[i]
    file_name = str(excel_dataframe['file_name'].iloc[i])
    mod_str = f"{file_name}-area-[{area}]-schizont-[{sci_count}]"
    excel_dataframe['modified_file'].iloc[i] = mod_str

100%|██████████| 442/442 [00:00<00:00, 5830.53it/s]


## 5. Moving it to directories

In [90]:
src_dir = os.getcwd()
print(src_dir)
os.mkdir('object_det_2')

/home/kanaad/Desktop/Strathclyde/Project/IchMLP/HHP/simulation/cropped pas stained images hhp - 2


In [91]:
dest_dir = src_dir+"/object_det_2"

for i in tqdm(range(len(excel_dataframe))):
    try:   
        file_to_copy = excel_dataframe['file_name'].iloc[i] + ".jpg"
        src_file = os.path.join(src_dir,file_to_copy)
        shutil.copy(src_file,dest_dir)

        dst_file = os.path.join(dest_dir,file_to_copy)
        renamed_file = excel_dataframe['modified_file'].iloc[i] + ".jpg"
        new_dst_file_name = os.path.join(dest_dir,renamed_file)

        os.rename(dst_file, new_dst_file_name)
        os.chdir(dest_dir)
    except Exception as e:
        except_file = excel_dataframe['file_name'].iloc[i]
        print(f'Exception occoured at file {except_file}')
        pass


100%|██████████| 442/442 [00:00<00:00, 1590.12it/s]

Exception occoured at file 14-181
Exception occoured at file 14-195
Exception occoured at file 14-199
Exception occoured at file 14-7175
Exception occoured at file 14-2176


## 6. Removing infrequently occouring data

In [92]:
frequencies = excel_dataframe['count'].value_counts().to_dict()

## 7. Creating a new data frame to hold all the new info

In [93]:
new_df = pd.DataFrame(columns=['file_name','count'],index=range(len(excel_dataframe)))

In [94]:
for i in tqdm(range(len(excel_dataframe))):
    val = excel_dataframe['count'].iloc[i]
    if frequencies[val] > 1:
        new_df['count'].iloc[i] = val
        new_df['file_name'].iloc[i] = excel_dataframe['modified_file'].iloc[i]
    else:
        pass
    

100%|██████████| 442/442 [00:00<00:00, 3795.23it/s]


In [95]:
new_df.dropna(inplace=True)

## 8. Creating train / test DataFrames

In [96]:
X = new_df['file_name'].to_numpy()
y = new_df['count'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X ,y ,test_size=0.1,stratify=y)

In [97]:
train = pd.DataFrame(columns=['file_name','count'],index=range(len(X_train)))
test = pd.DataFrame(columns=['file_name','count'],index=range(len(X_test)))

In [98]:
for i in tqdm(range(len(test))):
    test['file_name'].iloc[i] = X_test[i]
    test['count'].iloc[i] = y_test[i]

100%|██████████| 42/42 [00:00<00:00, 3726.06it/s]


In [99]:
for i in tqdm(range(len(train))):
    train['file_name'].iloc[i] = X_train[i]
    train['count'].iloc[i] = y_train[i]

100%|██████████| 369/369 [00:00<00:00, 4125.30it/s]


In [104]:
os.mkdir('train')
os.mkdir('test')

## 9. Moving all the files to train / test folders

In [105]:
src_dir = os.getcwd()

In [106]:
dest_dir = src_dir+"/train"

for i in tqdm(range(len(train))):
    try:   
        file_to_copy = train['file_name'].iloc[i] + ".jpg"
        src_file = os.path.join(src_dir,file_to_copy)
        shutil.copy(src_file,dest_dir)
        os.chdir(dest_dir)
    except Exception as e:
        except_file = train['file_name'].iloc[i]
        print(f'Exception occoured at file {except_file}')
        pass


100%|██████████| 369/369 [00:00<00:00, 1517.35it/s]

Exception occoured at file 14-2176-area-[2006]-schizont-[2]
Exception occoured at file 14-7175-area-[1955]-schizont-[1]
Exception occoured at file 14-181-area-[20617]-schizont-[34]
Exception occoured at file 14-199-area-[24488]-schizont-[27]


In [107]:
dest_dir = src_dir+"/test"

for i in tqdm(range(len(test))):
    try:   
        file_to_copy = test['file_name'].iloc[i] + ".jpg"
        src_file = os.path.join(src_dir,file_to_copy)
        shutil.copy(src_file,dest_dir)
        os.chdir(dest_dir)
    except Exception as e:
        except_file = test['file_name'].iloc[i]
        print(f'Exception occoured at file {except_file}')
        print(e)
        pass

100%|██████████| 42/42 [00:00<00:00, 1466.77it/s]

Exception occoured at file 14-195-area-[6581]-schizont-[3]
[Errno 2] No such file or directory: '/home/kanaad/Desktop/Strathclyde/Project/IchMLP/HHP/simulation/cropped pas stained images hhp - 2/object_det_2/14-195-area-[6581]-schizont-[3].jpg'
